## Call Notebook to Setup Environment and Initial Imports


In [1]:
%run "Setup_Env.ipynb"

## Mini-Project 1: Review Analyst

You are building an AI system to be able to look at customer reviews and do some complex analysis. for each review get ChatGPT to do the following:

  - Summarize the review. The summary should be at most 3 lines.
  - Highlight both the positives and negatives
  - Display the overall sentiment of the review (positive, negative, neutral)
  - Display a list of 3 - 5 emotions expressed by the customer in the review
  - If the sentiment is positive or neutral write an email and thank them for the review
  - If the sentiment is negative apologize and write an email with an appropriate response

Try to get the response in a nice structured format using an output parser

### Why Both Partial Variable AND Parser Are Needed

**The Flow:**
1. **Partial Variable** (`format_instructions`) → Gets injected into prompt → **Tells LLM how to respond**
2. **LLM** → Generates text response following those instructions
3. **Parser** → Converts that text response → **Into Python object**

**Think of it like this:**
- `format_instructions` = \"Please fill out this form in JSON format\"  
- `parser` = Takes the completed form and creates a structured object from it

**Without partial variable:** LLM wouldn't know how to structure its response  
**Without parser:** You'd get unstructured text instead of a clean Python object


### Access Customer Reviews

In [3]:
reviews = [
    f"""
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    """,
    f"""
    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.
    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,
    enhancing my gaming experience significantly. Price-wise, it's quite competitive,
    and I feel like I got a good deal. The delivery was swift, and it came well-protected,
    ensuring no damage during transport.
    """,
    f"""
    Ordered a set of wireless earbuds for running, and they've been a letdown.
    The sound constantly cuts out, and the fit is uncomfortable after only a few minutes of use.
    They advertised a 12-hour battery life, but I'm barely getting four hours.
    Considering the cost, I expected better quality and performance.
    They did arrive on time, but the positives end there. I'm already looking into a return.
    """,
    f"""
    The tablet stand I bought was touted as being sturdy and adjustable,
    but it's anything but. It wobbles with the slightest touch,
    and the angles are not holding up as promised. It feels like a breeze could knock it over.
    It was also pricier than others I've seen, which adds to the disappointment.
    It did arrive promptly, but what's the use if the product doesn't meet basic expectations?
    """,
    f"""
    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    """
]

### Define Output Parser

In [23]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# Define your desired data structure - like a python data class.
class ReviewAnalysisResponse(BaseModel):
    summary: str = Field(description="A brief summary of the customer review with maximum 3 lines")
    positives: list = Field(description="A list showing the positives mentioned by the customer in the review if any - max 3 points")
    negatives: list = Field(description="A list showing the negatives mentioned by the customer in the review if any - max 3 points")
    sentiment: str = Field(description="One word showing the sentiment of the review - positive, negative or neutral")
    emotions: list = Field(description="A list of 3 - 5 emotions expressed by the customer in the review")
    email: str = Field(description="Detailed email to the customer based on the sentiment")

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=ReviewAnalysisResponse)
format_instructions = parser.get_format_instructions()

### Create the input prompt for the LLM

In [43]:
# create the final prompt with formatting instructions from the parser
prompt_txt = """
             Analyze the given customer review below and generate the response based on the instructions
             mentioned below in the format instructions.
             Also remember to write a detailed email response for the email field based on these conditions:
               - email should be addressed to Dear Customer and signed with Service Agent
               - thank them if the review is positive or neutral
               - apologize if the review is negative

             Format Instructions:
             {format_instructions}

             Review:
             {review}
            """
prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=["review"],
    partial_variables={"format_instructions": format_instructions},
)

# Using PromptTemplate.from_template
# prompt = PromptTemplate.from_template(prompt_txt).partial(format_instructions=format_instructions)

### Create a LCEL LLM Chain

In [44]:
# create a simple LCEL chain to take the prompt, pass it to the LLM, enforce response format using the parser
chain = (prompt
           |
         chatgpt
           |
         parser)

### Format the input reviews

In [55]:
reviews_formatted = [{'review': review} for review in reviews]
print(reviews_formatted[0])

{'review': "\n    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.\n    The sound quality is impressively clear with just the right amount of bass.\n    It's also waterproof, which tested true during a recent splashing incident.\n    Though it's compact, the volume can really fill the space.\n    The price was a bargain for such high-quality sound.\n    Shipping was also on point, arriving two days early in secure packaging.\n    "}


### Get responses from the LLM

In [56]:
responses = chain.map().invoke(reviews_formatted)

In [47]:
responses[0].dict()

/var/folders/8v/xkrl1q210t5_4t4hvbx286800000gp/T/ipykernel_72675/632561186.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  responses[0].dict()


{'summary': 'The customer is highly satisfied with the Bluetooth speaker, praising its sound quality, waterproof feature, and value for money. They also appreciated the prompt shipping.',
 'positives': ['Fantastic sound quality with clear audio and good bass',
  'Waterproof feature works well',
  'Compact size with powerful volume'],
 'negatives': [],
 'sentiment': 'positive',
 'emotions': ['satisfaction', 'happiness', 'excitement'],
 'email': 'Dear Customer,\n\nThank you for your wonderful review! We are thrilled to hear that you are enjoying your new Bluetooth speaker and that it has met your expectations in sound quality, waterproof capability, and overall value. Your satisfaction is our top priority, and we appreciate your feedback on the prompt shipping as well.\n\nIf you have any further questions or need assistance, please feel free to reach out.\n\nBest regards,\nService Agent'}

In [48]:
responses[1].dict()

/var/folders/8v/xkrl1q210t5_4t4hvbx286800000gp/T/ipykernel_72675/409517893.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  responses[1].dict()


{'summary': 'The customer is highly satisfied with their new gaming keyboard, praising its responsiveness, backlighting, and competitive pricing. They also appreciated the swift delivery and protective packaging.',
 'positives': ['Responsive keys with a satisfying click',
  'Vibrant LED colors enhancing gaming experience',
  'Competitive pricing and good deal'],
 'negatives': [],
 'sentiment': 'positive',
 'emotions': ['satisfaction', 'excitement', 'contentment'],
 'email': "Dear Customer,\n\nThank you for your wonderful review! We are thrilled to hear that you are enjoying your new gaming keyboard and that it has met your expectations in terms of responsiveness and backlighting. It's great to know that you found the pricing competitive and that the delivery was swift and secure.\n\nWe appreciate your feedback and hope you continue to have an excellent gaming experience with your new keyboard!\n\nBest regards,\nService Agent"}

In [49]:
responses[2].dict()

/var/folders/8v/xkrl1q210t5_4t4hvbx286800000gp/T/ipykernel_72675/4100429645.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  responses[2].dict()


{'summary': 'The customer is disappointed with the wireless earbuds due to sound issues, uncomfortable fit, and poor battery life. They are considering a return.',
 'positives': ['Arrived on time'],
 'negatives': ['Sound cuts out', 'Uncomfortable fit', 'Poor battery life'],
 'sentiment': 'negative',
 'emotions': ['disappointment', 'frustration', 'dissatisfaction'],
 'email': 'Dear Customer,\n\nThank you for your feedback regarding the wireless earbuds. We sincerely apologize for the issues you have experienced with the sound quality, fit, and battery life. Your satisfaction is important to us, and we understand how disappointing it can be when a product does not meet your expectations. \n\nIf you would like assistance with the return process or if there is anything else we can do to help, please do not hesitate to reach out. \n\nThank you for bringing this to our attention.\n\nBest regards,\nService Agent'}

In [50]:
responses[3].dict()

/var/folders/8v/xkrl1q210t5_4t4hvbx286800000gp/T/ipykernel_72675/2247425992.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  responses[3].dict()


{'summary': "The customer is disappointed with the tablet stand's stability and adjustability, feeling it does not meet expectations despite prompt delivery.",
 'positives': ['Arrived promptly'],
 'negatives': ['Wobbles with the slightest touch',
  'Angles do not hold up as promised',
  'Pricier than others'],
 'sentiment': 'negative',
 'emotions': ['disappointment', 'frustration', 'dissatisfaction'],
 'email': "Dear Customer,\n\nThank you for your feedback regarding the tablet stand. We sincerely apologize for the disappointment you've experienced with its stability and adjustability. Your concerns are important to us, and we strive to ensure our products meet the highest standards. If you would like to discuss this further or explore options for a return or exchange, please feel free to reach out.\n\nThank you for bringing this to our attention.\n\nBest regards,\nService Agent"}

In [51]:
for response in responses:
  for k,v in response.dict().items():
    print(f'{k}:\n{v}')
  print('-----'*50)
  print('\n')

summary:
The customer is highly satisfied with the Bluetooth speaker, praising its sound quality, waterproof feature, and value for money. They also appreciated the prompt shipping.
positives:
['Fantastic sound quality with clear audio and good bass', 'Waterproof feature works well', 'Compact size with powerful volume']
negatives:
[]
sentiment:
positive
emotions:
['satisfaction', 'happiness', 'excitement']
email:
Dear Customer,

Thank you for your wonderful review! We are thrilled to hear that you are enjoying your new Bluetooth speaker and that it has met your expectations in sound quality, waterproof capability, and overall value. Your satisfaction is our top priority, and we appreciate your feedback on the prompt shipping as well.

If you have any further questions or need assistance, please feel free to reach out.

Best regards,
Service Agent
---------------------------------------------------------------------------------------------------------------------------------------------

/var/folders/8v/xkrl1q210t5_4t4hvbx286800000gp/T/ipykernel_72675/2504484312.py:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for k,v in response.dict().items():


In [62]:
for response in responses:
    print(response)

summary='The customer is highly satisfied with the Bluetooth speaker, praising its sound quality, waterproof feature, and value for money. They also appreciated the prompt shipping.' positives=['Fantastic sound quality with clear audio and good bass', 'Waterproof feature works effectively', 'Compact size with powerful volume'] negatives=[] sentiment='positive' emotions=['satisfaction', 'happiness', 'excitement'] email="Dear Customer,\n\nThank you for your wonderful review! We're thrilled to hear that you are enjoying your new Bluetooth speaker and that it has met your expectations in terms of sound quality, waterproof capability, and value for money. We also appreciate your feedback on the prompt shipping.\n\nIf you have any further questions or need assistance, feel free to reach out. Enjoy your beach outings with your fantastic speaker!\n\nBest regards,\nService Agent"
summary='The customer is highly satisfied with their new gaming keyboard, praising its responsiveness, backlighting,

In [52]:
import pandas as pd

pd.DataFrame(response.dict() for response in responses)

/var/folders/8v/xkrl1q210t5_4t4hvbx286800000gp/T/ipykernel_72675/1175396765.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  pd.DataFrame(response.dict() for response in responses)


,summary,positives,negatives,sentiment,emotions,email
0,The customer is highly satisfied with the Blue...,[Fantastic sound quality with clear audio and ...,[],positive,"[satisfaction, happiness, excitement]","Dear Customer,\n\nThank you for your wonderful..."
1,The customer is highly satisfied with their ne...,"[Responsive keys with a satisfying click, Vibr...",[],positive,"[satisfaction, excitement, contentment]","Dear Customer,\n\nThank you for your wonderful..."
2,The customer is disappointed with the wireless...,[Arrived on time],"[Sound cuts out, Uncomfortable fit, Poor batte...",negative,"[disappointment, frustration, dissatisfaction]","Dear Customer,\n\nThank you for your feedback ..."
3,The customer is disappointed with the tablet s...,[Arrived promptly],"[Wobbles with the slightest touch, Angles do n...",negative,"[disappointment, frustration, dissatisfaction]","Dear Customer,\n\nThank you for your feedback ..."
4,The customer had a negative experience with th...,[],"[Struggles with tougher foods, Incredibly nois...",negative,"[frustration, disappointment, anger]","Dear Customer,\n\nThank you for taking the tim..."


In [60]:
# Convert the list of responses to a pandas DataFrame
df = pd.DataFrame([response.model_dump() for response in responses])
df


,summary,positives,negatives,sentiment,emotions,email
0,The customer is highly satisfied with the Blue...,[Fantastic sound quality with clear audio and ...,[],positive,"[satisfaction, happiness, excitement]","Dear Customer,\n\nThank you for your wonderful..."
1,The customer is highly satisfied with their ne...,"[Responsive keys with a satisfying click, Vibr...",[],positive,"[satisfaction, excitement, contentment]","Dear Customer,\n\nThank you for your wonderful..."
2,The customer is disappointed with the wireless...,[Arrived on time],"[Sound cuts out, Uncomfortable fit, Poor batte...",negative,"[disappointment, frustration, dissatisfaction]","Dear Customer,\n\nThank you for your feedback ..."
3,The customer is disappointed with the tablet s...,[Arrived promptly],"[Wobbles with the slightest touch, Angles do n...",negative,"[disappointment, frustration, dissatisfaction]","Dear Customer,\n\nThank you for your feedback ..."
4,The customer had a negative experience with th...,[],"[Struggles with tougher foods, Incredibly nois...",negative,"[frustration, disappointment, anger]","Dear Customer,\n\nThank you for taking the tim..."


In [61]:
pd.DataFrame(responses)

,0,1,2,3,4,5
0,"(summary, The customer is highly satisfied wit...","(positives, [Fantastic sound quality with clea...","(negatives, [])","(sentiment, positive)","(emotions, [satisfaction, happiness, excitement])","(email, Dear Customer,\n\nThank you for your w..."
1,"(summary, The customer is highly satisfied wit...","(positives, [Responsive keys with a satisfying...","(negatives, [])","(sentiment, positive)","(emotions, [satisfaction, excitement, contentm...","(email, Dear Customer,\n\nThank you for your w..."
2,"(summary, The customer is disappointed with th...","(positives, [Arrived on time])","(negatives, [Sound cuts out, Uncomfortable fit...","(sentiment, negative)","(emotions, [disappointment, frustration, dissa...","(email, Dear Customer,\n\nThank you for your f..."
3,"(summary, The customer is disappointed with th...","(positives, [Arrived promptly])","(negatives, [Wobbles with the slightest touch,...","(sentiment, negative)","(emotions, [disappointment, frustration, dissa...","(email, Dear Customer,\n\nThank you for your f..."
4,"(summary, The customer had a negative experien...","(positives, [])","(negatives, [Struggles with tougher foods, Inc...","(sentiment, negative)","(emotions, [frustration, disappointment, anger])","(email, Dear Customer,\n\nThank you for taking..."
